## Pandas analysis

This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a couple of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

### 1. Create a Pandas DataFrame reading N rows of the 'data_000637.txt' dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k.

In [ ]:
import pandas as pd
df=pd.read_csv('data_000637.txt',delimiter=',')[0:12000]
df

### 2. Find out the number of BX in a ORBIT (the value 'x').

In [ ]:
df['BX_COUNTER'].count()

### 3. Find out how much the data taking lasted. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading out the whole dataset.

In [ ]:
import time
stattime=0
endtime=0
starttime=time.time()
df=pd.read_csv('data_000637.txt',delimiter=',')
endtime=time.time()
print(endtime-starttime)

### 4. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information).

In [ ]:
import time
stattime=0
endtime=0
starttime=time.time_ns()
df=pd.read_csv('data_000637.txt',delimiter=',')
endtime=time.time_ns()
print(endtime-starttime)

### 5. Replace the values (all 1) of the HEAD column randomly with 0 or 1.

In [ ]:
import numpy as np
for i in range(len(df['HEAD'])):
    df['HEAD'][i]=np.random.randint(2)
print(df['HEAD'].iloc[0:12000])

### 6. Create a new DataFrame that contains only the rows with HEAD=1.

In [ ]:
df['newDataFrame']=df['HEAD']==1
print(df)

### 7. Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel

In [ ]:
import matplotlib.pyplot as plt
for i in range(20):
    plt.bar(i,df['FPGA'][i])
plt.figure()
for i in range(20):
    plt.bar(i,df['TDC_CHANNEL'][i])

### 8. Use the groupby method to find out the noisy channels, i.e. the TDC channels with most counts (say the top 3)

In [ ]:
new_df=pd.value_counts(df['TDC_CHANNEL'])
new_df.iloc[0:3]

### 9. Count the number of unique orbits. Count the number of unique orbits with at least one measurement from TDC_CHANNEL=139

In [ ]:
print(len(df[df['TDC_CHANNEL'] == 139]))
print(df['TDC_CHANNEL'].unique())